In [29]:
import pandas as pd
data = pd.read_csv('/Users/aashnakanuga/Desktop/CDSS Hackathon/Data/Outputdata.csv')
data['topic_id'] = data['topic'].factorize()[0]
data

,topic,content,topic_id
0,Foreign Policy,word peace found common greeting shalom muslim...,0
1,Healthcare,trump administration taking swift action comba...,1
2,Infra,verge new technological revolution could impro...,2
3,Immigration,enact law write executive order reform broken ...,3
4,Foreign Policy,president donald trump helped kick session uni...,0
5,Immigration,tuesday department justice filed lawsuit state...,3
6,Infra,visit detroit last president trump announced a...,2
7,Foreign Policy,first lady melania trump joined president dona...,0
8,Infra,first man moon held american astronaut moon ma...,2
9,Healthcare,latest estimate show american died drug involv...,1


In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', stop_words='english')
features = tfidf.fit_transform(data.content).toarray()
labels = data.topic_id
features.shape

(65, 596)

In [31]:
topic_id_df = data[['topic', 'topic_id']].drop_duplicates().sort_values('topic_id')
topic_to_id = dict(topic_id_df.values)
id_to_topic = dict(topic_id_df[['topic_id', 'topic']].values)

In [32]:
from sklearn.feature_selection import chi2
import numpy as np
N = 10
for topic, topic_id in sorted(topic_to_id.items()):
    features_chi2 = chi2(features, labels == topic_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    print("# '{}':".format(topic))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))

# 'Agriculture':
  . Most correlated unigrams:
. area
. rural
. natural
. hand
. assistance
. received
. helping
. infrastructure
. aid
. response
# 'Budget and spending':
  . Most correlated unigrams:
. eliminate
. management
. economy
. saving
. growth
. budget
. small
. cut
. business
. tax
# 'Economy and Jobs':
  . Most correlated unigrams:
. provides
. practice
. place
. market
. favor
. export
. worker
. china
. trade
. product
# 'Education':
  . Most correlated unigrams:
. presidential
. event
. advisor
. memorandum
. college
. science
. computer
. school
. education
. student
# 'Energy and Environment':
  . Most correlated unigrams:
. natural
. imposed
. development
. power
. domestic
. source
. fuel
. rule
. environmental
. energy
# 'Foreign Policy ':
  . Most correlated unigrams:
. civil
. iran
. forward
. south
. meeting
. middle
. conflict
. thank
. regime
. peace
# 'Healthcare ':
  . Most correlated unigrams:
. recovery
. crisis
. believe
. control
. latest
. treatment
. d

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
count_vect = CountVectorizer()
content_train_counts = count_vect.fit_transform(data['content'])
tfidf_transformer = TfidfTransformer()
content_train_tfidf = tfidf_transformer.fit_transform(content_train_counts)
clf = MultinomialNB().fit(content_train_tfidf, data['topic'])

In [60]:
import pandas as pd
test_data = pd.read_csv('/Users/aashnakanuga/Desktop/CDSS Hackathon/Data/clean3.csv')
print(test_data.shape)
category=[]
for i in range(len(test_data)):
    try:
        category.append(clf.predict(count_vect.transform([test_data['content'][i]]))[0])
    except Exception as e:
        print((([test_data['content'][i]])))
        print(i)
        category.append('No Keywords')
        
print(category)

(1335, 2)
[nan]
109
[nan]
201
[nan]
380
[nan]
444
[nan]
781
[nan]
998
[nan]
1013
[nan]
1030
[nan]
1044
[nan]
1073
[nan]
1100
[nan]
1126
[nan]
1189
[nan]
1251
[nan]
1256
[nan]
1257
[nan]
1258
[nan]
1259
['Foreign Policy ', 'Foreign Policy ', 'Budget and spending', 'Economy and Jobs', 'Healthcare ', 'Immigration ', 'Economy and Jobs', 'Budget and spending', 'Economy and Jobs', 'Economy and Jobs', 'Budget and spending', 'Law', 'Economy and Jobs', 'Immigration ', 'Budget and spending', 'Foreign Policy ', 'National Security', 'Foreign Policy ', 'Law', 'Law', 'Budget and spending', 'Agriculture', 'Agriculture', 'Economy and Jobs', 'Economy and Jobs', 'Energy and Environment', 'Energy and Environment', 'Budget and spending', 'Economy and Jobs', 'Economy and Jobs', 'Agriculture', 'Law', 'Law', 'Education', 'Immigration ', 'Immigration ', 'Education', 'Energy and Environment', 'Budget and spending', 'Foreign Policy ', 'Budget and spending', 'Social Program', 'Energy and Environment', 'Healthcar